In [1]:
from urllib import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [2]:

# all_data_df = pd.read_csv("all_data_1999_to_2017.csv")
# draft_data_df = pd.read_csv("draft_data_1999_to_2017.csv")
# predraft_measurement_df = pd.read_csv("nba-pre-draft-measurements.csv")
# blocks_ncaa_df = pd.read_csv("blocks_2002_to_2016.csv")
# steals_ncaa_df = pd.read_csv("steals_2002_to_2016.csv")
# scoring_ncaa_df = pd.read_csv("scoring_2002_to_2016.csv")
# assists_ncaa_df = pd.read_csv("assits_2002_to_2016.csv")
# rebounds_ncaa_df = pd.read_csv("rebounds_2002_to_2016.csv")

blocks_ncaa_df = pd.read_csv("blocks_2017_to_2018.csv")
steals_ncaa_df = pd.read_csv("steals_2017_to_2018.csv")
scoring_ncaa_df = pd.read_csv("scoring_2017_to_2018.csv")
assists_ncaa_df = pd.read_csv("assits_2017_to_2018.csv")
rebounds_ncaa_df = pd.read_csv("rebounds_2017_to_2018.csv")


In [3]:
assists_ncaa_df.head()
pattern = r'(.*), (\w)'

assists_ncaa_df['PLAYER'] = assists_ncaa_df.apply(lambda row: re.search(pattern, row['PLAYER']).group(1), axis = 1)
blocks_ncaa_df['PLAYER'] = blocks_ncaa_df.apply(lambda row: re.search(pattern, row['PLAYER']).group(1), axis = 1)
steals_ncaa_df['PLAYER'] = steals_ncaa_df.apply(lambda row: re.search(pattern, row['PLAYER']).group(1), axis = 1)
scoring_ncaa_df['PLAYER'] = scoring_ncaa_df.apply(lambda row: re.search(pattern, row['PLAYER']).group(1), axis = 1)
rebounds_ncaa_df['PLAYER'] = rebounds_ncaa_df.apply(lambda row: re.search(pattern, row['PLAYER']).group(1), axis = 1)

# Rename Rank
assists_ncaa_df.rename(columns={'RK':'RK_AST'}, inplace=True)
blocks_ncaa_df.rename(columns={'RK':'RK_BLK'}, inplace=True)
steals_ncaa_df.rename(columns={'RK':'RK_STL'}, inplace=True)
scoring_ncaa_df.rename(columns={'RK':'RK_SCO'}, inplace=True)
rebounds_ncaa_df.rename(columns={'RK':'RK_REB'}, inplace=True)

assists_ncaa_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
blocks_ncaa_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
steals_ncaa_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
scoring_ncaa_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
rebounds_ncaa_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
# print assists_ncaa_df.head()
# print blocks_ncaa_df.head()
# print steals_ncaa_df.head()
# print scoring_ncaa_df.head()
# print rebounds_ncaa_df.head()


In [4]:
assists_ncaa_df['TOPG'] = assists_ncaa_df['TO'] / assists_ncaa_df['GP']
assists_ncaa_df['APG'] = assists_ncaa_df['AST'] / assists_ncaa_df['GP']
assists_ncaa_df['AST/TO'] = assists_ncaa_df['AST'] / assists_ncaa_df['TO']

steals_ncaa_df['STPG'] = steals_ncaa_df['STL'] / steals_ncaa_df['GP']
steals_ncaa_df['TOPG'] = steals_ncaa_df['TO'] / steals_ncaa_df['GP']
steals_ncaa_df['ST/TO'] = steals_ncaa_df['STL'] / steals_ncaa_df['TO']
steals_ncaa_df['ST/PF'] = steals_ncaa_df['STL'] / steals_ncaa_df['PF']

blocks_ncaa_df['BLKPG'] = blocks_ncaa_df['BLK'] / blocks_ncaa_df['GP']
blocks_ncaa_df['BLK/PF'] = blocks_ncaa_df['BLK'] / blocks_ncaa_df['PF']

scoring_ncaa_df['FGA'] = scoring_ncaa_df['FGM-FGA'].apply(lambda x: re.search(r'(.*)-(.*)', x).group(2)).astype(np.float)
scoring_ncaa_df['3PA'] = scoring_ncaa_df['3PM-3PA'].apply(lambda x: re.search(r'(.*)-(.*)', x).group(2)).astype(np.float)
scoring_ncaa_df['FTA'] = scoring_ncaa_df['FTM-FTA'].apply(lambda x: re.search(r'(.*)-(.*)', x).group(2)).astype(np.float)
scoring_ncaa_df.drop(['FGM-FGA', '3PM-3PA', 'FTM-FTA'], axis = 1, inplace = True)

In [5]:
def mergeTwoDf(df1, df2):
    # Find columns intersection as merge keys
    cols_df1 = set(df1.columns)
    cols_df2 = set(df2.columns)
    mergeKeys = list(cols_df1.intersection(cols_df2))
    return  pd.merge(df1, df2, on = mergeKeys, how = 'outer')

# Merge all dataframes
df_all = mergeTwoDf(assists_ncaa_df, steals_ncaa_df)
df_list = [blocks_ncaa_df, scoring_ncaa_df, rebounds_ncaa_df]
for df in df_list:
    df_all = mergeTwoDf(df_all, df)
grouped = df_all.groupby('PLAYER')


# Compute means for a player of different years
df_list = []
for key, gp in grouped:    
    gp_mean = gp.mean()
    df_mean = gp_mean.reshape(1, 38)
    df_mean = pd.DataFrame(data = df_mean, columns = gp_mean.index)
    df_mean['PLAYER'] = key
    df_mean['TEAM'] = gp.iloc[0]['TEAM']
    df_list.append(df_mean)    
df_mean = pd.concat(df_list)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:20: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [6]:
df_mean_post = df_mean.drop(['AST', 'TO', 'AST/TO', 'YEAR', 'STL', 'STPG', 'ST/TO','ST/PF',
                             'BLK', 'BLKPG', 'BLK/PF', 'OFF', 'DEF', 'REB', 'RPG', 'APG'], axis = 1)
print len(df_mean_post.columns)
df_mean_post.columns

for col in df_mean_post.columns:
    if col[0:3] == 'RK_':
        df_mean_post[col].fillna(121, inplace = True)
    else:
        df_mean_post[col].fillna(0, inplace = True)
        
# df_mean_post[df_mean_post['PLAYER'] == 'Stephen Curry']
df_mean_post.reset_index(inplace = True)
df_mean_post.drop('index', axis = 1, inplace = True)
df_mean_post

24


,GP,MPG,TOPG,AP40M,RK_AST,STP40M,PF,RK_STL,BLKP40M,RK_BLK,...,RK_SCO,FGA,3PA,FTA,ORPG,DRPG,RP40,RK_REB,PLAYER,TEAM
0,2.0,31.5,0.000000,0.0,121.0,0.00,0.0,121.0,0.00,121.0,...,121.0,0.0,0.0,0.0,4.0,5.5,12.1,71.0,AJ Brodeur,PENN
1,4.0,23.8,0.000000,0.0,121.0,0.00,0.0,121.0,0.00,121.0,...,121.0,0.0,0.0,0.0,1.5,7.3,14.7,98.0,Abdul Lewis,NJIT
2,3.0,28.7,0.000000,0.0,121.0,0.00,9.0,121.0,3.26,56.0,...,121.0,0.0,0.0,0.0,0.0,0.0,0.0,121.0,Abdul-Malik Abu,NCST
3,3.0,35.3,1.666667,0.0,121.0,3.02,9.0,35.0,0.00,121.0,...,121.0,0.0,0.0,0.0,0.0,0.0,0.0,121.0,Ahmad Thomas,UNCA
4,4.0,35.3,0.000000,0.0,121.0,0.00,0.0,121.0,0.00,121.0,...,12.0,20.3,7.3,7.8,0.0,0.0,0.0,121.0,Alec Peters,VALP
5,3.0,31.0,3.333333,10.3,9.0,0.00,0.0,121.0,0.00,121.0,...,121.0,0.0,0.0,0.0,0.0,0.0,0.0,121.0,Alec Wintering,PORT
6,2.0,26.5,0.000000,8.3,68.0,0.00,0.0,121.0,0.00,121.0,...,121.0,0.0,0.0,0.0,0.0,0.0,0.0,121.0,Alex Copeland,YALE
7,3.0,29.3,2.000000,7.3,76.0,3.18,5.0,73.0,0.00,121.0,...,121.0,0.0,0.0,0.0,0.0,0.0,0.0,121.0,Alex Robinson,TCU
8,3.0,29.0,1.333333,0.0,121.0,3.22,2.0,87.0,0.00,121.0,...,121.0,0.0,0.0,0.0,0.0,0.0,0.0,121.0,Alterique Gilbert,CONN
9,3.0,31.0,1.666667,7.7,42.0,3.44,3.0,29.0,0.00,121.0,...,121.0,0.0,0.0,0.0,0.0,0.0,0.0,121.0,Amadi Udenyi,PEPP


In [8]:
def separateNameField(row):
    pattern = r'(\w*[\'-]*\w* \w*[\'-]*\w*).*'
    s = row['Player']
    s = s.replace('\xd5', '\'')
    m = re.search(pattern, s)
    row['Player'] = m.group(1)
    return row

allstar_stats = pd.read_csv('allstar.csv')
allstar_stats = allstar_stats.apply(separateNameField, axis = 1)
player_name = set(allstar_stats['Player'])
mean_set = set(df_mean_post['PLAYER'])

df_mean_post['allstar'] = False
for i, r in df_mean_post.iterrows():
    df_mean_post.loc[i, 'allstar'] = r['PLAYER'] in player_name
df_stars_T = df_mean_post[df_mean_post['allstar'] == True]
df_stars_F = df_mean_post[df_mean_post['allstar'] == False]

df_stars_T.to_csv("ncaa_allstar_2016_data_T.csv")
df_stars_F.to_csv("ncaa_allstar_2016_data_F.csv")


In [9]:
# def splitPermute(X, threshold):
#     t_ind = X.index
#     t_ind_perm = np.random.permutation(len(t_ind))
#     perm = t_ind[t_ind_perm]

#     ind = t_ind[t_ind_perm]
#     cutoff = int(len(X) * threshold)
#     t_ind_3 = perm[:cutoff]
#     t_ind_7 = perm[cutoff:]
#     return X.loc[t_ind_3], X.loc[t_ind_7]

def splitPermute(X, cutoff):
    t_ind = X.index
    t_ind_perm = np.random.permutation(len(t_ind))
    perm = t_ind[t_ind_perm]

    ind = t_ind[t_ind_perm]
    t_ind_te = perm[:cutoff]
    t_ind_tr = perm[cutoff:]
    return X.loc[t_ind_te], X.loc[t_ind_tr]


Xt_te, Xt_tr = splitPermute(df_stars_T, 20)
Xf_te, Xf_tr = splitPermute(df_stars_F, 20)


X_te = pd.concat([Xt_te, Xf_te])
X_tr = pd.concat([Xt_tr, Xf_tr])
y_te = X_te['allstar']
y_tr = X_tr['allstar']
X_te.drop(['PLAYER', 'TEAM', 'allstar'], axis = 1, inplace = True)
X_tr.drop(['PLAYER', 'TEAM', 'allstar'], axis = 1, inplace = True)

In [17]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_tr, y_tr)

for threshold in range(1, 10, 1):
    threshold /= 10.0
    print threshold
    print "tr: ", ((regr.predict(X_tr) > threshold)  == y_tr).mean()
    print "te: ", ((regr.predict(X_te) > threshold) == y_te).mean()
    print "----"



NameError: name 'X_tr' is not defined

Index([u'TEAM_x', u'GP_x', u'MPG_x', u'PTS', u'FG%', u'3P%', u'FT%', u'FGA',
       u'3PA', u'FTA', u'NCAA_GP_x', u'NCAA_MPG_x', u'NCAA_PTS', u'NCAA_FG%',
       u'NCAA_3P%', u'NCAA_FT%', u'NCAA_FGA', u'NCAA_3PA', u'NCAA_FTA',
       u'PLAYER', u'TEAM_y', u'GP_y', u'MPG_y', u'ORPG', u'DRPG', u'RP40',
       u'NCAA_GP_y', u'NCAA_MPG_y', u'NCAA_ORPG', u'NCAA_DRPG', u'NCAA_RP40'],
      dtype='object')
TEAM
TEAM


KeyError: (u'TEAM_x', u'occurred at index TEAM_x')

                PLAYER All_Star
0  Kareem Abdul-Jabbar     True
1  Shareef Abdur-Rahim     True
2          Alvan Adams     True
3        Michael Adams     True
4         Mark Aguirre     True
  TEAM_x  GP_x  MPG_x  PTS_x  FG%_x  3P%_x  FT%_x   FGA  3PA  FTA   ...     \
0     VT  32.0   36.5   18.8  0.454  0.377  0.817  15.0  6.5  3.4   ...      
1   IONA  33.0   37.3   19.9  0.436  0.394  0.781  14.2  8.0  5.5   ...      
2    BUT  32.0   35.3   17.0  0.376  0.354  0.958  13.1  7.9  4.5   ...      
3    RMU  28.0   35.4   16.9  0.466  0.350  0.850  13.3  4.2  3.6   ...      
4    VAN  33.0   26.3   17.0  0.593  0.000  0.778   9.9  0.1  6.7   ...      

   2016_TEAM  2016_GP  2016_MPG  2016_PTS  2016_FGM-FGA  2016_FG%  \
0          0      0.0       0.0       0.0             0       0.0   
1          0      0.0       0.0       0.0             0       0.0   
2          0      0.0       0.0       0.0             0       0.0   
3          0      0.0       0.0       0.0             0       0

                  PLAYER
0    Kareem Abdul-Jabbar
1    Shareef Abdur-Rahim
2            Alvan Adams
3          Michael Adams
4           Mark Aguirre
5            Danny Ainge
6      LaMarcus Aldridge
7              Ray Allen
8         Kenny Anderson
9        Carmelo Anthony
10        Nate Archibald
11        Gilbert Arenas
12           Paul Arizin
13             Vin Baker
14       Charles Barkley
15         Don Barksdale
16          Dick Barnett
17         Leo Barnhorst
18           Dana Barros
19            Rick Barry
20          Elgin Baylor
21           Butch Beard
22           Ralph Beard
23           Zelmo Beaty
24          Walt Bellamy
25      Chauncey Billups
26             Dave Bing
27            Larry Bird
28         Otis Birdsong
29      Rolando Blackman
..                   ...
375            John Wall
376          Ben Wallace
377       Gerald Wallace
378      Rasheed Wallace
379         Paul Walther
380          Bill Walton
381         Bobby Wanzer
382    Kermit Washington


## Merge with all star stats

IOError: File ref/allstar.csv does not exist